### this is my notebook for trying to figure out how to make medical history information into something useful

#### first we import libraries and data

In [1]:
import pandas as pd
import numpy as np

import os
import re
from collections import Counter

import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import sklearn.metrics
import xgboost as xgb

import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import random as rn

from IPython.display import clear_output


In [2]:
#use this cell when working online
path = '/floyd/home/ed-triage'
data_path = '/floyd/home/data'

In [3]:
#use this cell when working from home
path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/ed-triage'
data_path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/data/ED triage project/combo'

In [4]:
clin = pd.read_csv(data_path + '/complete_clean_combo_data.csv', index_col = 0,low_memory = False)

In [5]:
clin.shape

(165433, 121)

#### then we load up my helper functions

In [6]:

def show_confusion_matrix2(confusion_matrix, labels):
    LABELS = labels

    plt.figure(figsize=(3, 3))
    sns.heatmap(confusion_matrix, cmap = 'Blues', xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d", annot_kws={"size": 15});
    plt.title("Confusion matrix", fontsize=10)
    plt.ylabel('True label', fontsize=10)
    plt.xlabel('Predicted label', fontsize=10)
    plt.show()
    
def get_metrics(preds , predictions, show_confusion = True, print_output = True, return_values = False):
    
    y_test = targets.numpy()
    
    f1_w = sklearn.metrics.f1_score(y_test, predictions, average='weighted')
    f1 = sklearn.metrics.f1_score(y_test, predictions, average=None)
    acc = sklearn.metrics.accuracy_score(y_test, predictions)
    prec = sklearn.metrics.precision_score(y_test,predictions, average=None) 
    rec = sklearn.metrics.recall_score(y_test,predictions, average=None)
    auroc = sklearn.metrics.roc_auc_score(y_test,preds[:,1], average=None)
    confusion = sklearn.metrics.confusion_matrix(y_test, predictions)
    
    tp, fn, fp, tn = confusion[0,0], confusion[0,1], confusion[1,0], confusion[1,1]
    
    sens = tp/(tp + fn)
    spec = tn/(tn + fp)
    ppv = tp/(tp + fp)
    npv = tn/(tn + fn)
    
    
    if print_output:
        print ('Metrics Report:')
        print ('---------------')
        print ('weighted f1: ', f1_w)
        print ('AUROC:       ',auroc)
        print ('accuracy:    ', acc)
        print ('precision:   ', prec)
        print ('recall:      ', rec)
        print ('sensitivity: ', sens)
        print ('specificity: ', spec)
        print ('PPV:         ', ppv)
        print ('NPV:         ', npv)
        
    if show_confusion:
        show_confusion_matrix2(confusion, labels = data.classes)
     
    if return_values == True:
        return f1_w, auroc, acc, sens, ppv



#### then we clean up the data in the medical history column

In [8]:
history = list(clin['MedicalHistory'].str[16:].astype('str'))

In [9]:
#this is to clean the text in the medical history column (a little bit)
def fixhx(s):
    s =s.replace('<LT><LF>','')
    s = s.replace('<LT>LF>','')
    s = s.replace('<LF><LF>','')
    s =s.replace('Pt.', 'patient')
    s =s.replace('Pt', 'patient')
    s = s.lower()
    s = s.replace('pt.', 'patient')
    s = s.replace('y/o', 'year old')
    s = s.replace('c/o', 'complains of')
    s = s.replace('h/o', 'history of')
    s = s.replace('sob', 'shortness of breath')
    s = s.replace('c/p', 'chest pain')
    s = s.replace('hrs', 'hours')
    s = s.replace('hx', 'history')
    s = s.replace('n/v', 'nausea and/or vomiting')
    s = s.replace('a/e', 'air entry')
    s = s.replace('a/o', 'alert and oriented')
    s = s.replace('a&o', 'alert and oriented')
    s = s.replace('d/c', 'discharge')
    s = s.replace('u/s', 'ultrasound')
    s = s.replace('yrs', 'years')
    s = s.replace('lmp', 'last menstrual period')
    s = s.replace('w/', 'with')
    s = s.replace('@', 'at ')
    s = s.replace('b/c', 'because')
    s = re.sub(r'\br\s*\/\s*o','rule out',s)
    s = re.sub(r'\bc\s*\/\s*o','complains of',s)
    s = re.sub(r'\bpt.','patient ',s)
    s = re.sub(r'\blt.','left ',s)
    s = re.sub(r'\brt.','right ',s)
    s = re.sub(r'\bmd.','doctor ',s)
    s = re.sub(r'\bra\b','room air ',s)
    s = re.sub(r'\bbp.','blood pressure ',s)
    s = re.sub(r'\bcp.','chest pain ',s)
    s = re.sub(r'\bfd.','family doctor ',s)
    s = s.replace('abx', 'antibiotics')
    s = s.replace(' htn ', ' hypertension ')
    s = re.sub(r'\btyl\b', 'tylenol', s)
    s = re.sub(r'\bprn\b', 'as needed ',s)
    s = re.sub(r'\bvag\b', 'vaginal ',s)
    s = re.sub(r'\bpo\b', 'by mouth ',s)
    s = re.sub(r'\bllq\b', 'left lower quadrant',s)
    s = re.sub(r'\brlq\b', 'right lower quadrant  ',s)
    s = re.sub(r'\bluq\b', 'left upper quadrant',s)
    s = re.sub(r'\bruq\b', 'right upper quadrant  ',s)
    s = re.sub(r'\buti\b', 'urinary tract infection ',s)
    s = re.sub(r'\brd', 'right digit number',s)
    s = re.sub(r'\bld', 'left digit number',s)
    s = re.sub(r'\br\b', 'right ',s)
    s = re.sub(r'\bl\b', 'left ',s)
    s = re.sub(r'\bed\b', 'emergency department ',s)
    s = re.sub(r'\ber\b', 'emergency room ',s)
    s = re.sub(r'\bbm\b', 'bowel movement ',s)
    s = re.sub('(?<=[a-zA-Z])-(?=[a-zA-Z])', ' ', s)
    s = re.sub(r'\bwic\b', 'walk in clinic ',s)
    s = re.sub(r'\bems\b', 'emergency medical services ',s)
    s = re.sub(r'\bmvc\b', 'motor vehicle collision  ',s)
    s = re.sub(r'\bgp\b', 'family doctor  ',s)
    s = re.sub(r'\bloc\b', 'loss of consciousness ',s)
    s = re.sub(r'\bsts\b', 'states  ',s)
    s = re.sub(r'\bmvc', 'motor vehicle collision ',s)
    s = re.sub(r'\bnsr\b', 'normal sinus rhythm ',s)
    s = s.replace("--", "-")
    s = s.replace('-', ' ')
    s = re.sub('and/or', 'or', s)
    s = re.sub(r'\bneg', 'negative', s)
    s = re.sub('s/b', 'sent by', s)
    s = re.sub(r'nausea\s*\/\s*vomiting', 'nausea or vomiting', s)
    s = re.sub(';',',',s)
    s = re.sub('aaox3', 'alert and oriented', s)
    s = re.sub(r'csm', ' circulation, sensation and movement ',s)
    s = re.sub(r'sz', ' seizure ',s)
    s = re.sub('hgb', 'hemoglobin', s)
    s = re.sub('\basa\b', 'aspirin', s)
    s = re.sub('\butd\b', 'up to date', s)
    s = re.sub('\brx\b', 'treatment', s)
    s = re.sub('\bnpo\b', 'nothing by mouth', s)
    
    punctuation_spacer = str.maketrans({key: f"{key} " for key in ".,"})
    s = s.translate(punctuation_spacer)
    
    s = re.sub(r'\s+,',',', s)
    s = re.sub(r'\s+\.','.', s)

    
    
    #remove extra white spaces
    s = re.sub(r'\s+', ' ', s)

    #note that what's missing here is removal of punctuation
    return s

In [10]:
replacements = (',','<LT>LF>', '.', ';')
drop_list = ('"', '?', 'had', 'has', 'have', 'left', 'right', 'sided','weeks', 'history of', 'h/o')   #added all but first 2 options may 30, 2020
split_hist = []
for item in history:
    for r in replacements:
        item = item.replace(r, '::')
        
    item = item.split('::')
    item = [re.sub(r'\(.*?\)\ *', '', s) for s in item]  #remove punctuation
    item = [re.sub(r'\b\d+\b', '', s) for s in item]  #remove numbers
    item = [re.sub(r'%', '', s) for s in item]  #remove percent sign
    item = [re.sub(r'\b[0-9]\w+''', '', s) for s in item]  #remove words that start with numbers
    item = [re.sub(r'^[^\w#]*', '',s) for s in item]    #remove leading symbols except fracture
    item = [re.sub(r'^-\s*', '',s) for s in item] #remove leading dashes
    item = [fixhx(s) for s in item]   #run the cleaning function above to get rid of some of the abbreviation
    
    for d in drop_list:
        item = [x.replace(d,'') for x in item]   #get rid of the words from the blacklist above
    
    item = [x.strip().lower() for x in item if x != 'nan']   #get rid of white spaces
    item = [x for x in item if len(x) > 2]
    item = list(filter(None, item))
    
    split_hist.append(item)
    
    
 #split hist is a list of lists, where each entry is the list of past medical history conditions for each patient

In [21]:
#split hist is a list of lists, where each entry is the list of past medical history conditions for each patient
split_hist[:10]

[['no significant medical history'],
 ['no significant medical history'],
 ['no significant medical history'],
 ['dialysis haemo/pd', 'iddm', 'crf', 'heart problem', 'high cholesterol'],
 ['fibromyalgia'],
 ['enlarged liver'],
 ['anxiety', 'autistic', 'adhd', 'depression', 'asthma', 'celiac'],
 ['no significant medical history'],
 ['hypothyroid'],
 ['anxiety']]

In [22]:
#my hideous way of limiting each diagnosis to 2 words long
split_hist2 = []
for item in split_hist:
    item2 = []
    for dx in item:
        if dx != 'no significant medical history':
            new_dx = ' '.join(dx.split()[:2])
            item2.append(new_dx)
        else:
            item2.append(dx)
    split_hist2.append(item2)

In [23]:
#I've processed it so each diagnosis is no more than 2 words long
split_hist2[:10]

[['no significant medical history'],
 ['no significant medical history'],
 ['no significant medical history'],
 ['dialysis haemo/pd', 'iddm', 'crf', 'heart problem', 'high cholesterol'],
 ['fibromyalgia'],
 ['enlarged liver'],
 ['anxiety', 'autistic', 'adhd', 'depression', 'asthma', 'celiac'],
 ['no significant medical history'],
 ['hypothyroid'],
 ['anxiety']]

### exploring different diagnoses

In [24]:
diagnoses = Counter()
for hx in split_hist2:   #note that this is done with my split modified split hist
    for item in hx:
        diagnoses[item] += 1

In [25]:
#there were 23721 different medical histories used in the dataset!!!
#now down to 16334
len(diagnoses)

15852

In [27]:
diagnoses.most_common(100)

[('no significant medical history', 73713),
 ('hypertension', 21662),
 ('high cholesterol', 19788),
 ('niddm', 12397),
 ('htn', 7743),
 ('asthma', 6976),
 ('high bp', 6403),
 ('depression', 6378),
 ('thyroid', 4936),
 ('anxiety', 4855),
 ('hypothyroid', 3981),
 ('iddm', 3803),
 ('gerd', 3412),
 ('diabetes', 3259),
 ('acid reflux', 2421),
 ('arthritis', 2221),
 ('anemia', 1708),
 ('copd', 1511),
 ('high chol', 1297),
 ('chf', 1175),
 ('schizophrenia', 1154),
 ('gout', 1122),
 ('seizure', 1060),
 ('kidney stones', 1056),
 ('cva', 1054),
 ('enlarged prostate', 1020),
 ('atrial fibrillation', 954),
 ('dementia', 924),
 ('cholesterol', 893),
 ('migraines', 874),
 ('adhd', 794),
 ('osteoporosis', 784),
 ('chronic back', 738),
 ('bipolar', 724),
 ('afib', 717),
 ('immunizations up', 688),
 ('smoker', 678),
 ('urinary tract', 632),
 ('cad', 624),
 ('fibromyalgia', 601),
 ('substance misuse', 597),
 ('breast cancer', 582),
 ('pneumonia', 562),
 ('chol', 546),
 ('pacemaker/cardioverter/defib', 5

In [29]:
sorted(diagnoses.keys())[500:550]

['alcohol used',
 'alcohol with',
 'alcohol withdrawal',
 'alcohol withdrawl',
 'alcoholic',
 'alcoholic cirrhosis',
 'alcoholic cirrohisis',
 'alcoholic hepatiits',
 'alcoholic hepatitis',
 'alcoholic liver',
 'alcoholic seizure',
 'alcoholic withdrawal',
 'alcoholism',
 'alcoholism over',
 'alendronate',
 'alert and',
 'alesse',
 'aleve',
 'alfusocin',
 'alheimers',
 'alhocolism',
 'ali dec',
 'alkalosis',
 'alkylosing spondiolosis',
 'all',
 'all tests',
 'allard cardiologist',
 'allergic eczema',
 'allergic reaction',
 'allergic reactions',
 'allergic rhinitis',
 'allergic rxn',
 'allergic rxns',
 'allergic to',
 'allergies',
 'allergy',
 'allergy induced',
 'allergy related',
 'allergy to',
 'allery to',
 'allopurinol',
 'allurinipol',
 'almesartan',
 'almost lost',
 'alocohol abuse',
 'alocoholism',
 'alopacia',
 'alopecia',
 'aloprecia',
 'alpha']

### this is getting much better.  gonne now do some work to make all the aaa's, c-sections, etc. collapse to a single diagnosis

In [30]:
#pick all the diagnoses from the counter that start with c (no particular reason for that letter)
c_words = [word for word in sorted(diagnoses.keys()) if word[0] == 'c']

In [31]:
len(c_words)

1572

In [33]:
#lots of spelling errors in here
c_words[10:20]

['c sction',
 'c sec',
 'c sect',
 'c section',
 'c section/',
 'c sections',
 'c spine',
 'c/section x2',
 'c1 c8',
 'c1 fracture']

In [34]:
c_counter = Counter(c_words); len(c_counter)

1572

In [35]:
#what happens if you make all the items with the same first word are considered the same?
first_words = []
for word in c_counter.keys():
    first_word =  word.split()[0]
    first_words.append(first_word) 

In [36]:
first_words[20:30]

['c2', 'c2', 'c3', 'c4', 'c5', 'c5', 'c5/c6', 'c5c6', 'c6', 'c6']

In [37]:
#this is promising.  decreases the number of diagnoses in half
fw_counter = Counter(first_words); len(fw_counter)

646

In [39]:
#this function takes in a counter with word frequencies of similar words
#it returns the most common spelling as the "best spelling"

def get_best_spelling(dict_name):
    max_value = -10
    best_spelling = 0
    for k,v in dict_name.items():
        if v > max_value:
            max_value = v
            best_spelling = k
    
    return best_spelling 

In [40]:
#this is stolen from peter norvigs spellchecker
import re
from collections import Counter

#def words(text): return re.findall(r'\w+', text.lower())

WORDS = diagnoses

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

#### now i need to find a way to generate the frequency dict/counter to use to get the best spelling

In [41]:
#gonna start using the functions from peter norvig
#arbitrarily using one of my c-words:  'cataracts'

def poss_words(word):
    return list(set(list(known(edits1(word))) +  list(known(edits2(word))) + [word]))

In [43]:
print (poss_words('c-section'))
print (poss_words('c section'))
print (poss_words('cataract'))
print (poss_words('crohns'))
print(poss_words('c-diff'))

['csection', 'resection', 'c-section', 'disection']
['csection', 'c sction', 'c section', 'c cesection', 'disection', 'c sections', 'resection']
['cateract', 'catracts', 'caatracts', 'catract', 'catarac', 'catarct', 'cataract', 'catarract', 'cataracts']
['cons', 'crohn', 'cronhs', 'chrons', 'croups', 'chrohns', 'crohns']
['cdif', 'diff', 'cdiff', 'cdiffe', 'c-diff']


In [44]:
c_spell_counter = {}
for first_word in first_words: #[20:200]:
    if len(first_word) > 4:
        poss_spells = poss_words(first_word)
        word_count = dict()
        for spell in poss_spells:
            word_count[spell] = fw_counter[spell]
        c_spell_counter[first_word] = word_count

In [45]:
for k,v in c_spell_counter.items():
    print (k,':')
    print (v)
    print (get_best_spelling(v))
    print ()

c/section :
{'csection': 2, 'resection': 0, 'c/section': 1, 'disection': 0}
csection

c5/c6 :
{'c5/c6': 1}
c5/c6

ca/home/app#meds :
{'ca/home/app#meds': 1}
ca/home/app#meds

caatracts :
{'catracts': 1, 'caatracts': 1, 'catract': 2, 'catacts': 1, 'cataract': 14, 'cataracts': 4}
cataract

cabg1999 :
{'cabg1999': 1}
cabg1999

cabg\ :
{'cabgx': 2, 'cabge': 1, 'cabg\\': 1, 'cabd': 1, 'cagb': 1, 'cbag': 1, 'cabg': 17, 'cabgg': 1}
cabg

cabge :
{'cabgx': 2, 'cabge': 1, 'cabd': 1, 'cagb': 1, 'cbag': 1, 'cabg': 17, 'cabgg': 1}
cabg

cabgg :
{'cabgx': 2, 'cabge': 1, 'cabd': 1, 'cagb': 1, 'cbag': 1, 'cabg': 17, 'cabgg': 1}
cabg

cabgx :
{'cabgx': 2, 'cabge': 1, 'cabd': 1, 'cagb': 1, 'cbag': 1, 'cabg': 17, 'cabgg': 1}
cabg

cabgx3 :
{'cabgx': 2, 'cabge': 1, 'cabgx3': 3, 'cabg': 17, 'cabgg': 1}
cabg

cad/ashd :
{'cad/ashd': 1}
cad/ashd

cad/bypass :
{'cad/bypass': 1}
cad/bypass

cadiomyopathy :
{'cariomyopathy': 1, 'cardiomypathy': 2, 'cardiomyopathy': 2, 'cardiomyopaty': 1, 'cadiomyopathy': 1}
ca

ciprodex :
{'ciprodex': 2, 'cipralex': 1}
ciprodex

ciprofloxacin :
{'ciprofloxacin': 1}
ciprofloxacin

circlulation :
{'circlulation': 1}
circlulation

circudosis :
{'circudosis': 1}
circudosis

circulation :
{'circulation': 4}
circulation

circumcision :
{'circumcision': 4}
circumcision

cirhosis :
{'cirhosis': 1, 'kiphosis': 0, 'cirrohsis': 1, 'cirrohosis': 1, 'chirrhosis': 1, 'cirrhosis': 4}
cirrhosis

cirrhosis :
{'cirrohosis': 1, 'cirrohsis': 1, 'cirrhosis': 4, 'chirrhosis': 1}
cirrhosis

cirrhosys :
{'cirrohsis': 1, 'cirrohosis': 1, 'chirrhosis': 1, 'cirrhosis': 4, 'cirrhosys': 1}
cirrhosis

cirrohosis :
{'cirrohosis': 1, 'cirrohsis': 1, 'chirrhosis': 1, 'cirrhosis': 4}
cirrhosis

cirrohsis :
{'cirrohosis': 1, 'cirrohsis': 1, 'chirrhosis': 1, 'cirrhosis': 4}
cirrhosis

citlopram :
{'citlopram': 1}
citlopram

cj=holesterol :
{'cholesterol': 15, 'cj=holesterol': 1}
cholesterol

claims :
{'claims': 1}
claims

clarithromycin :
{'clarithromycin': 1}
clarithromycin

clausterphobic :
{

costochondritis :
{'costochondritis': 1, 'costochondriittis': 1, 'costcochondritis': 1}
costochondritis

cough :
{'mouth': 0, 'cough': 7}
cough

cough/cold :
{'cough/cold': 1}
cough/cold

could :
{'colds': 1, 'coprd': 1, 'old': 0, 'wound': 0, 'cold': 4, 'coils': 1, 'coil': 1, 'could': 1, 'coli': 1, 'cough': 7, 'copd': 10}
copd

couldn't :
{"couldn't": 1}
couldn't

coumadin :
{'coumadin': 1}
coumadin

course :
{'toure': 0, 'course': 1}
course

coversion :
{'coversion': 1}
coversion

coverstyl :
{'coversyl': 1, 'coverstyl': 1}
coversyl

coversyl :
{'coversyl': 1, 'coverstyl': 1}
coversyl

crack :
{'back': 0, 'crack': 8, 'trach': 0}
crack

crack) :
{'crack': 8, 'crack)': 1}
crack

crack/cocaine :
{'crack/cocaine': 1}
crack/cocaine

cracked :
{'cracked': 2, 'crack': 8}
crack

cracking :
{'cracking': 1}
cracking

crainiotomy :
{'craniotomy': 3, 'crainiotomy': 2}
craniotomy

cranial :
{'cranial': 5}
cranial

craniotomy :
{'craniotomy': 3, 'crainiotomy': 2}
craniotomy

crdiac :
{'cardaic': 2,

In [46]:
# a little shorter, because this won't contain any of the very short words, abbreviations
len(c_spell_counter)

566

### combining all of the above into a single function

the function takes the diagnoses counter and generates a dictionary of possible spellings for each first word > 5 letters, beginning with the chosen letter

In [1241]:
def first_letter_counter(letter, diagnosis_counter = diagnoses, word_length=5):
    collected_words = [word for word in sorted(diagnosis_counter.keys()) if word[0] == letter]
    
    let_counter = Counter(collected_words)
    
    first_words = []
    for word in let_counter.keys():
        first_word =  word.split()[0]
        first_words.append(first_word) 
        
    fw_counter = Counter(first_words)
    
    spell_dict = {}
    for first_word in first_words:
        if len(first_word) > word_length:
            poss_spells = poss_words(first_word)
            word_count = dict()
            for spell in poss_spells:
                word_count[spell] = fw_counter[spell]
            spell_dict[first_word] = word_count
        
    return collected_words, spell_dict

In [1248]:
#this should have the same results as the cell by cell implementation with the letter c
%time c_collected_words, c_spell_dict = first_letter_counter('c', diagnoses, 4)

CPU times: user 2min 49s, sys: 392 ms, total: 2min 49s
Wall time: 2min 51s


In [1249]:
for k,v in c_spell_counter.items():
    print (k,':')
    print (v)
    print (get_best_spelling(v))
    print ()

c-anca :
{'c-anca': 1}
c-anca

c-cestionsx :
{'c-cestionsx': 1}
c-cestionsx

c-diff :
{'cdiff': 3, 'c-diff': 5, 'cdiffe': 1, 'cdif': 1, 'diff': 0}
c-diff

c-pap :
{'c-pap': 2, 'bipap': 0}
c-pap

c-sction :
{'c-section': 21, 'c-sections': 1, 'csection': 2, 'c-sction': 1}
c-section

c-sec :
{'c-sec': 1, 'losec': 0}
c-sec

c-sect :
{'c-sect': 3, 'c-sec': 1}
c-sect

c-section :
{'disection': 0, 'resection': 0, 'c-sections': 1, 'c-section': 21, 'csection': 2, 'c-sction': 1}
c-section

c-section-born :
{'c-section-born': 1}
c-section-born

c-section-no :
{'c-section-no': 1}
c-section-no

c-sections :
{'c-section': 21, 'c-sections': 1, 'csection': 2, 'c-sction': 1}
c-section

c-spine :
{'spine': 0, 'c-spine': 3}
c-spine

c/section :
{'csection': 2, 'disection': 0, 'resection': 0, 'c/section': 1}
csection

c1-c8 :
{'c1-c8': 1}
c1-c8

c3-c6 :
{'c3-c6': 1}
c3-c6

c4-c5 :
{'c4-c5': 1}
c4-c5

c5-c6 :
{'c5-c6': 2}
c5-c6

c5/c6 :
{'c5/c6': 1}
c5/c6

caatracts :
{'catract': 2, 'catracts': 1, 'catarac

{'cirrohosis': 1, 'cirrhosis': 4, 'chirrhosis': 1, 'cirrohsis': 1}
cirrhosis

cirrohsis :
{'cirrohosis': 1, 'cirrhosis': 4, 'chirrhosis': 1, 'cirrohsis': 1}
cirrhosis

citlopram :
{'citlopram': 1}
citlopram

cj=holesterol :
{'cj=holesterol': 1, 'cholesterol': 15}
cholesterol

claims :
{'claims': 1}
claims

clarithromycin :
{'clarithromycin': 1}
clarithromycin

clausterphobic :
{'claustrophobic': 1, 'clausterphobic': 1}
claustrophobic

claustophobia :
{'claustrophobic': 1, 'claustrophobia': 1, 'claustophobia': 1}
claustrophobic

claustrophobia :
{'claustrophobic': 1, 'claustrophobia': 1, 'claustophobia': 1}
claustrophobic

claustrophobic :
{'claustrophobic': 1, 'clausterphobic': 1, 'claustrophobia': 1, 'claustophobia': 1}
claustrophobic

clavalin :
{'clavalin': 1, 'clavulin': 1}
clavalin

clavicle :
{'clavicle': 1}
clavicle

clavicular :
{'clavicular': 1}
clavicular

clavulin :
{'clavalin': 1, 'clavulin': 1}
clavalin

clear :
{'cleared': 1, 'ucler': 0, 'cream': 1, 'ear': 0, 'clear': 1, 

In [1354]:
#c_spell_dict should be the same as c_spell_counter above
#c_collected_words should be the same as c_words above
len(c_spell_dict), len (c_collected_words)

(618, 1637)

In [1251]:
c_spell_dict == c_spell_counter, c_collected_words == c_words

(True, True)

looking good.  let's try with another letter for fun

In [1253]:
%time h_collected_words, h_spell_dict = first_letter_counter('h', diagnoses, 5)

CPU times: user 1min 55s, sys: 237 ms, total: 1min 55s
Wall time: 1min 55s


In [1254]:
print(len(h_collected_words))
print(len(h_spell_dict))

1177
431


In [1257]:
for k,v in h_spell_dict.items():
    print (k,':')
    print (v)
    print (get_best_spelling(v))
    print ()

h-plyorie :
{'h-pylorie': 1, 'h-pyloric': 1, 'h-plyorie': 1, 'h-pylori': 2}
h-pylori

h-pylori :
{'h-plyorie': 1, 'h-pyloric': 1, 'h-pylorie': 1, 'h-pylori': 2, 'hpylori': 2, 'pylori': 0}
h-pylori

h-pyloric :
{'h-plyorie': 1, 'h-pyloric': 1, 'h-pylorie': 1, 'h-pylori': 2, 'hpylori': 2}
h-pylori

h-pylorie :
{'h-plyorie': 1, 'h-pyloric': 1, 'h-pylorie': 1, 'h-pylori': 2, 'hpylori': 2}
h-pylori

haalluciations :
{'haalluciations': 1, 'hallucinations': 3}
hallucinations

haemangioma :
{'hemangioma': 3, 'haemangioma': 1}
hemangioma

haemhorroids :
{'hemmorroids': 1, 'hemorroids': 1, 'haemhorroids': 1, 'haemorrhoids': 1}
hemmorroids

haemodialysis :
{'hemodailysis': 1, 'hemodialysis': 8, 'haemodialysis': 1, 'hemodilaysis': 1}
hemodialysis

haemophilia :
{'haemophilia': 1, 'hemophillia': 1, 'hemophilia': 3}
hemophilia

haemorrhoid :
{'haemorrhoid': 2, 'hemmorrhoid': 1, 'haemorrhoids': 1, 'hemorroid': 2, 'hemorrhoid': 3, 'hemorhoid': 1, 'hemorrhoids': 5}
hemorrhoids

haemorrhoids :
{'hemorrh


hydromorphone :
{'hydromorphone': 1}
hydromorphone

hydronephorsis :
{'hydroneprosis': 1, 'hydrnephrosis': 1, 'hydronephrosis': 2, 'hydronephorsis': 1}
hydronephrosis

hydronephrosis :
{'hydrnephrosis': 1, 'hydronephrosis': 2, 'hydroneprosis': 1, 'hydronephorsis': 1}
hydronephrosis

hydroneprosis :
{'hydrnephrosis': 1, 'hydronephrosis': 2, 'hydroneprosis': 1, 'hydronephorsis': 1}
hydronephrosis

hydronitis :
{'hydronitis': 1, 'hyderinitis': 1}
hydronitis

hydrophalus :
{'hydrocephalus': 3, 'hydrophalus': 1}
hydrocephalus

hydroxycorquine :
{'hydroxycorquine': 1}
hydroxycorquine

hydroxyquin :
{'hydroxyquin': 1}
hydroxyquin

hyeperemesis :
{'hyperemesis': 6, 'hypremesis': 1, 'hyeperemesis': 1, 'hyperemisis': 1}
hyperemesis

hyermobile :
{'hyermobile': 1}
hyermobile

hyertension :
{'hypertension': 6, 'hyptertension': 1, 'hypertention': 2, 'hyertension': 1, 'hpertension': 1, 'hyoertension': 1, 'hypertenstion': 1, 'hypotension': 1, 'hyptension': 1, 'hypetension': 1, 'hypertesion': 1}
hype

In [1258]:
#gonna copy the first stage of collected words and overwrite the new words with correct spellings
h_collected_words_old = h_collected_words.copy()

In [1259]:
#this will go through a list of diagnoses, pull out the first word, see if the first word is in the
#dictionary fo alternate spellings for a word and replace it with the best spelling

for i in range(len(h_collected_words)):
    word_list = h_collected_words[i].split()
    if word_list[0] in h_spell_dict.keys():
        best_word = get_best_spelling(h_spell_dict[word_list[0]])
        if word_list[0] != best_word: 
            word_list[0] = best_word
    
    h_collected_words[i] = ' '.join(word_list)
   

In [1260]:
len(h_collected_words)

1177

In [1355]:
#this is the same length as the dict above but the first word has been corrected in many cases
h_collected_words[:30]

['h -',
 'h piloro',
 'h pylorie',
 'h pylorie',
 'h pylorie',
 'h pylorie',
 'h-pylori',
 'h-pylori',
 'h-pylori',
 'h-pylori completed',
 'h1n1',
 'ha d/t',
 'hear loss',
 'hair procedure',
 'hairline fracture',
 'hard of',
 'haliperdol',
 'hallucinations',
 'hallucinations --nyd',
 'hallucinations post',
 'hamilton hall',
 'hamstring strain',
 'hamstring tear',
 'hamstring tear',
 'hand',
 'hand amputee',
 'htn and',
 'hand burn',
 'hand fracture',
 'hand infection-finished']

In [1262]:
#but it hasnt shortened that much
len(Counter(h_collected_words)), len(Counter(h_collected_words_old))

(1010, 1177)

running the function again to see if it changes the corrections at all...

In [1263]:
h_counter = (Counter(h_collected_words))
%time h_collected_words, h_spell_dict = first_letter_counter('h', h_counter, 5)

CPU times: user 1min 28s, sys: 392 ms, total: 1min 29s
Wall time: 1min 31s


In [1264]:
#and it doesnt
len(Counter(h_collected_words))

1010

In [1265]:
for k,v in h_spell_dict.items():
    print (k,':')
    print (v)
    print (get_best_spelling(v))
    print ()

h-pylori :
{'h-plyorie': 0, 'h-pyloric': 0, 'h-pylorie': 0, 'h-pylori': 2, 'hpylori': 2, 'pylori': 0}
h-pylori

hairline :
{'hairline': 1}
hairline

haliperdol :
{'haliperdol': 1}
haliperdol

hallucinations :
{'haalluciations': 0, 'hallucinations': 3, 'hallucination': 0}
hallucinations

hamilton :
{'hamilton': 1}
hamilton

hamstring :
{'hamstring': 3}
hamstring

hand/foot/mouth :
{'hand/foot/mouth': 1}
hand/foot/mouth

harada :
{'harada': 1}
harada

haringtons :
{'haringtons': 1}
haringtons

hartmann :
{'hartmann': 1}
hartmann

head/neck :
{'head/neck': 1}
head/neck

headache :
{'earache': 0, 'headache': 6, 'headaches': 0}
headache

headache-seen :
{'headache-seen': 1}
headache-seen

headinjury :
{'headinjury': 1}
headinjury

heaing :
{'heaing': 1}
heaing

healed :
{'healed': 2}
healed

healthy :
{'healthy': 4}
healthy

hearing :
{'hearing': 20}
hearing

heartattack :
{'heartattack': 1}
heartattack

heartburn :
{'heartburn': 1}
heartburn

helico :
{'helico': 1}
helico

hemachromatosis 

now we are going to collapse the diagnoses to only care about the first word...

In [1267]:
h_first_words = []
for word in h_counter.keys():
    first_word =  word.split()[0]
    h_first_words.append(first_word) 
        
h_fw_counter = Counter(h_first_words)

In [1358]:
#much shorter but a lot of information is lost
len(h_fw_counter)

313

In [1359]:
h_fw_counter.most_common()[100:120]

[('hdhd', 1),
 ('head/neck', 1),
 ('headache-seen', 1),
 ('headinjury', 1),
 ('heard', 1),
 ('heartattack', 1),
 ('heartburn', 1),
 ('heat', 1),
 ('hel', 1),
 ('helico', 1),
 ('hellp', 1),
 ('hemochromatosis', 1),
 ('hemacro', 1),
 ('hemangioendothelioma', 1),
 ('hematocrit', 1),
 ('hemi-laminectomy', 1),
 ('hemiclitomy', 1),
 ('hemidiaphragm', 1),
 ('hemifacial', 1),
 ('hemihyperplasia', 1)]

In [1360]:
sorted(h_counter.most_common()[100:120])

[('hbh', 1),
 ('hbv', 1),
 ('hcl', 1),
 ('hctz', 1),
 ('hd tuethusat', 1),
 ('hdhd', 1),
 ('he a', 1),
 ('he does', 1),
 ('head ache', 1),
 ('head and', 1),
 ('head concusion', 1),
 ('head fracture', 1),
 ('head injjury', 1),
 ('head injuries', 1),
 ('head injury', 1),
 ('head injury/', 1),
 ('head pain', 1),
 ('head surgery', 1),
 ('head sx', 1),
 ('heart faillure', 1)]

now we are going to see if we can correct whole diagnoses

In [1273]:
poss_words('h pylori')

['h pylorie',
 'hp pyloric',
 'pylori',
 'hel pylori',
 'h pyloric',
 'h pylori',
 'hpylori',
 'h piloro',
 'h plyori']

In [1274]:
poss_words('hysterectomy')

['hyperectomy',
 'hysterectomy',
 'hysterctomy',
 'histerectomy',
 'hyterectmomy',
 'hysterectomuy',
 'hystprectomy',
 'hyserectomy',
 'hysterrectomy',
 'hestorectomy',
 'hystrectomy',
 'hyterectomy']

In [1279]:
poss_words('heart attack')

['heaart attack',
 'heart attacked',
 'heart attack',
 'hert attack',
 'heart attacks',
 'heartattack',
 'hear attack']

very promising as it finds lots of similar diagnoses

In [1281]:
def whole_diagnosis_counter(diagnosis_counter = diagnoses):
    
    #collected_words = [word for word in sorted(diagnosis_counter.keys()) if word[0] == letter]
    
    spell_dict = {}
    for diagnosis in diagnosis_counter.keys():
        poss_spells = poss_words(diagnosis)
        word_count = dict()
        for spell in poss_spells:
            word_count[spell] = fw_counter[spell]
        spell_dict[first_word] = word_count
        
    return spell_dict

In [1286]:
list(h_counter.keys())[:20]

['h -',
 'h piloro',
 'h plyori',
 'h pylori',
 'h pyloric',
 'h pylorie',
 'h- pylori',
 'h- pyloric',
 'h-pylori',
 'h-pylori completed',
 'h1n1',
 'ha d/t',
 'hallucinations',
 'hemangioma',
 'hemmorroids',
 'hemodialysis',
 'hemophilia',
 'hemorrhoids',
 'hemorrhoids problem',
 'hair loss']

In [1361]:
def whole_diagnosis_counter(diagnosis_counter = diagnoses):
    
    spell_dict = {}
    for diagnosis in list(diagnosis_counter.keys()):
        if len(diagnosis)>5:
            poss_spells = poss_words(diagnosis)
            word_count = dict()
            for spell in poss_spells:
                word_count[spell] = diagnosis
                _counter[spell]
            spell_dict[diagnosis] = word_count

In [1317]:
spell_dict = whole_diagnosis_counter(h_counter); spell_dict

{'h piloro': {'h piloro': 1, 'h pylori': 1},
 'h plyori': {'h pylorie': 1,
  'h pyloric': 1,
  'hpylori': 1,
  'h pylori': 1,
  'h plyori': 1},
 'h pylori': {'h pylorie': 1,
  'hp pyloric': 1,
  'pylori': 0,
  'hel pylori': 1,
  'h pyloric': 1,
  'h pylori': 1,
  'hpylori': 1,
  'h piloro': 1,
  'h plyori': 1},
 'h pyloric': {'h pylorie': 1,
  'hp pyloric': 1,
  'h pyloric': 1,
  'hpylori': 1,
  'h pylori': 1,
  'h plyori': 1},
 'h pylorie': {'h pylorie': 1,
  'hp pyloric': 1,
  'h pyloric': 1,
  'hpylori': 1,
  'h pylori': 1,
  'h plyori': 1},
 'h- pylori': {'h pylorie': 1,
  'hp pyloric': 1,
  'h-pyloric': 0,
  'h-pylorie': 0,
  'hel pylori': 1,
  'h- pyloric': 1,
  'h-pylori': 4,
  'h- pylori': 1,
  'h pyloric': 1,
  'h pylori': 1,
  'hpylori': 1,
  'h plyori': 1},
 'h- pyloric': {'h pylorie': 1,
  'hp pyloric': 1,
  'h-pyloric': 0,
  'h-pylorie': 0,
  'h- pyloric': 1,
  'h-pylori': 4,
  'h- pylori': 1,
  'h pyloric': 1,
  'h pylori': 1},
 'h-pylori': {'h-plyorie': 0,
  'h-pyloric':

In [1318]:
for k,v in spell_dict.items():
    print (k,':')
    print (v)
    print (get_best_spelling(v))
    print ()

h piloro :
{'h piloro': 1, 'h pylori': 1}
h piloro

h plyori :
{'h pylorie': 1, 'h pyloric': 1, 'hpylori': 1, 'h pylori': 1, 'h plyori': 1}
h pylorie

h pylori :
{'h pylorie': 1, 'hp pyloric': 1, 'pylori': 0, 'hel pylori': 1, 'h pyloric': 1, 'h pylori': 1, 'hpylori': 1, 'h piloro': 1, 'h plyori': 1}
h pylorie

h pyloric :
{'h pylorie': 1, 'hp pyloric': 1, 'h pyloric': 1, 'hpylori': 1, 'h pylori': 1, 'h plyori': 1}
h pylorie

h pylorie :
{'h pylorie': 1, 'hp pyloric': 1, 'h pyloric': 1, 'hpylori': 1, 'h pylori': 1, 'h plyori': 1}
h pylorie

h- pylori :
{'h pylorie': 1, 'hp pyloric': 1, 'h-pyloric': 0, 'h-pylorie': 0, 'hel pylori': 1, 'h- pyloric': 1, 'h-pylori': 4, 'h- pylori': 1, 'h pyloric': 1, 'h pylori': 1, 'hpylori': 1, 'h plyori': 1}
h-pylori

h- pyloric :
{'h pylorie': 1, 'hp pyloric': 1, 'h-pyloric': 0, 'h-pylorie': 0, 'h- pyloric': 1, 'h-pylori': 4, 'h- pylori': 1, 'h pyloric': 1, 'h pylori': 1}
h-pylori

h-pylori :
{'h-plyorie': 0, 'h-pyloric': 0, 'h-pylorie': 0, 'h-pylori': 4

heart attack

hernia hernia :
{'hertial hernia': 0, 'hernia hernia': 1}
hernia hernia

herpes hernia :
{'hertus hernia': 0, 'herpes hernia': 1}
herpes hernia

hysterectomy :
{'hyperectomy': 0, 'hysterectomy': 12, 'hysterctomy': 1, 'histerectomy': 0, 'hyterectmomy': 0, 'hysterectomuy': 0, 'hystprectomy': 0, 'hyserectomy': 0, 'hysterrectomy': 0, 'hestorectomy': 0, 'hystrectomy': 0, 'hyterectomy': 0}
hysterectomy

hfm disease :
{'hfm disease': 1, 'gum disease': 0}
hfm disease

hgb deficiency :
{'iga deficiency': 0, 'hgb deficiency': 1, 'b deficiency': 0}
hgb deficiency

hgh bp :
{'hgh bp': 1, 'high b': 1, 'hihg bp': 1, 'high bp': 1, 'high bs': 1}
hgh bp

hgh cholesterol :
{'high cholestrol': 1, 'high choletserol': 1, 'hogh cholesterol': 1, 'hi cholesterol': 1, 'high cholestyrol': 1, 'high cholestero': 1, 'high choelsterol': 1, 'high cholesteraol': 1, 'high cholesetrol': 1, 'high cbholesterol': 1, 'high cholesteroll': 1, 'high chpolesterol': 1, 'high cholelsterol': 1, 'high sholesterol': 1


hmf forte :
{'hmf forte': 1}
hmf forte

hodegkins :
{'hodegkins': 1}
hodegkins

hodegkins lymphoma :
{'hodkgins lymphoma': 0, 'hogkins lymphoma': 0, 'hodgkin lymphoma': 1, 'hodkins lymphoma': 0, 'hodgkins lymphoma': 1, 'hodegkins lymphoma': 4}
hodegkins lymphoma

hodgeskin lipoma :
{'hodgeskin lipoma': 1}
hodgeskin lipoma

hodgkin lymphoma :
{'hodkgins lymphoma': 0, 'hogkins lymphoma': 0, 'hodgkin lymphoma': 1, 'hodgkin lymptoma': 1, 'hodkins lymphoma': 0, 'hodgkins lymphoma': 1}
hodgkin lymphoma

hodgkin lymptoma :
{'hodgkins lymphoma': 1, 'hodgkin lymphoma': 1, 'hodgkin lymptoma': 1}
hodgkins lymphoma

hodgkin's lymphoma :
{"hogkin's lymphoma": 1, "hodgkin's lymphome": 1, 'hodkgins lymphoma': 0, 'hogkins lymphoma': 0, 'hodgkin lymphoma': 1, 'hodkins lymphoma': 0, 'hodgkins lymphoma': 1, "hodgkin's lymphoma": 1, "hodgekin's lymphoma": 0}
hogkin's lymphoma

hodgkin's lymphome :
{'hodgkins lymphone': 1, "hodgkin's lymphome": 1, "hodgekin's lymphoma": 0, 'hodgkins lymphoma': 1, "hodgkin

In [1319]:
h_collected_words_old = h_collected_words.copy()

In [1363]:
len(h_collected_words), len(set(h_collected_words))

(1010, 699)

In [1322]:
h_collected_words[:10]

['h -',
 'h piloro',
 'h plyori',
 'h pylori',
 'h pyloric',
 'h pylorie',
 'h- pylori',
 'h- pyloric',
 'h-pylori',
 'h-pylori completed']

In [1362]:
sorted(spell_dict.keys())[:10]

['h piloro',
 'h plyori',
 'h pylori',
 'h pyloric',
 'h pylorie',
 'h- pylori',
 'h- pyloric',
 'h-pylori',
 'h-pylori completed',
 'ha d/t']

In [1327]:
#this is my function for overwriting diagnoses with their correct spellings
for i in range(len(h_collected_words)):
    try:
        h_collected_words[i] = get_best_spelling(spell_dict[h_collected_words[i]])
    except:
        continue
    


In [1365]:
h_collected_words[:20]

['h -',
 'h piloro',
 'h pylorie',
 'h pylorie',
 'h pylorie',
 'h pylorie',
 'h-pylori',
 'h-pylori',
 'h-pylori',
 'h-pylori completed',
 'h1n1',
 'ha d/t',
 'hear loss',
 'hair procedure',
 'hairline fracture',
 'hard of',
 'haliperdol',
 'hallucinations',
 'hallucinations --nyd',
 'hallucinations post']

In [1364]:
len(set(h_collected_words))

699

### more progress here.  need to refine the best spelling function so if there's a tie, it doesn't change the original spelling

In [1330]:
poss_words('heavy menstration')

['heavy menstration', 'heavy menstruation', 'heavy menstruution']

In [1331]:
spell_dict['heavy menstration']

{'heavy menstration': 1, 'heavy menstruation': 1, 'heavy menstruution': 1}

In [1332]:
spell_dict['heavy menstruation']

{'heavy menstration': 1, 'heavy menstruation': 1, 'heavy menstruution': 1}

In [1334]:
poss_words('helico bacteria')

['helico bacteria']

In [1335]:
poss_words('hemicolectomy -')

['hemicolectomy -', 'hemicolectomy']

In [1337]:
spell_dict['hemicolectomy -']

{'hemicolectomy -': 1, 'hemicolectomy': 1}

In [1338]:
new_h_counter = Counter(h_collected_words)

In [1340]:
new_h_counter['hemicolectomy -']

1

In [1342]:
new_h_counter['hemochromatosis']

3

In [1349]:
spell_dict['h pylorie']

{'h pylorie': 1,
 'hp pyloric': 1,
 'h pyloric': 1,
 'hpylori': 1,
 'h pylori': 1,
 'h plyori': 1}

In [1348]:
new_h_counter['h pylorie']

5

In [1350]:
new_h_counter['h pylori']

0

In [1351]:
new_h_counter['h-pylori']

3

In [1353]:
poss_words('h pylori')

['h pylorie',
 'hp pyloric',
 'pylori',
 'hel pylori',
 'h pyloric',
 'h pylori',
 'hpylori',
 'h piloro',
 'h plyori']

In [1343]:
def first_word_counter(counter):
    first_words = []
    for word in counter.keys():
        first_word =  word.split()[0]
        first_words.append(first_word) 

    fw_counter = Counter(first_words)
    
    return fw_counter

In [1344]:
h_fw_counter = first_word_counter(new_h_counter)

In [1345]:
len(h_fw_counter)

313

In [1346]:
h_fw_counter.most_common()

[('heart', 61),
 ('high', 39),
 ('hip', 31),
 ('history', 27),
 ('htn', 20),
 ('hernia', 15),
 ('hysterectomy', 13),
 ('hearing', 11),
 ('hypertension', 10),
 ('hand', 9),
 ('hep', 9),
 ('hepatic', 9),
 ('heavy', 8),
 ('hemiplegia', 7),
 ('hiv', 7),
 ('head', 6),
 ('hemorrhoids', 6),
 ('herniated', 6),
 ('home', 6),
 ('headache', 5),
 ('heel', 5),
 ('hypertrophic', 5),
 ('healthy', 4),
 ('hemangioma', 4),
 ('hereditary', 4),
 ('herpes', 4),
 ('hypo', 4),
 ('holter', 4),
 ('hormonal', 4),
 ('hormone', 4),
 ('hyperactive', 4),
 ('hyperplasia', 4),
 ('h', 3),
 ('hear', 3),
 ('hallucinations', 3),
 ('hemodialysis', 3),
 ('hematuria', 3),
 ('hepatitis', 3),
 ('hgb', 3),
 ('hodgkins', 3),
 ('hospitalized', 3),
 ('htnn', 3),
 ('huntingtons', 3),
 ('hydrocele', 3),
 ('hydrocephalus', 3),
 ('hyperemesis', 3),
 ('hypoglycemia', 3),
 ('h-pylori', 2),
 ('hamstring', 2),
 ('he', 2),
 ('healed', 2),
 ('hematoma', 2),
 ('hemicolectomy', 2),
 ('hemophilia', 2),
 ('hemorrhagic', 2),
 ('her', 2),
 ('her

In [1347]:
def get_best_spelling2(dict_name):
    max_value = -10
    best_spelling = 0
    for k,v in dict_name.items():
        if v > max_value:
            max_value = v
            best_spelling = k
    
    return best_spelling 